In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
import bs4 as bs
import nltk
from nltk.tokenize import sent_tokenize
import re
from nltk.stem import PorterStemmer
from nltk.tag import pos_tag
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics 
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('labelled130.csv')

In [ ]:
df['sentiment'][0:130].isna().sum()

In [ ]:
df.head()

In [ ]:
df.dropna(subset=['sentiment'], inplace=True)

In [ ]:
df.reset_index(drop=True, inplace=True)
df

In [ ]:
df.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1'], inplace=True)

In [ ]:
df.tail()

In [ ]:
X = df.iloc[:,-2]
X.head()

In [ ]:
#Codes refered from class notebooks
from nltk.corpus import stopwords


def email_cleaner(email):
    
    #1. Remove HTML tags
    email = bs.BeautifulSoup(email).text
    
    #2. Use regex to find emoticons
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', email)
    
    #3. Remove punctuation
    email = re.sub("[^a-zA-Z]", " ",email)
    
    #4. Tokenize into words (all lower case)
    email = email.lower().split()
    
    #5. Remove stopwords
    eng_stopwords = set(stopwords.words("english"))
    email = [w for w in email if not w in eng_stopwords]
    
    #6. Join the review to one sentence
    email = ' '.join(email+emoticons)
    # add emoticons to the end

    return(email)

In [ ]:
for i in range(len(X)):
    X[i] = email_cleaner(X[i])
    if (i%10 == 0):
        print ("Finished: " + str(i))

In [ ]:
X

In [ ]:
Y = df.iloc[:,-1]
Y.head()

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# put everything together in a function

def predict_sentiment(X, Y):

    print("Creating the bag of words model!\n")
    # CountVectorizer" is scikit-learn's bag of words tool, here we show more keywords 
    vectorizer = CountVectorizer(analyzer = "word",   \
                                 tokenizer = None,    \
                                 preprocessor = None, \
                                 stop_words = None,   \
                                 max_features = 2000) 
    
    X_train, X_test, y_train, y_test = train_test_split(\
    X, Y, random_state=0, test_size=.2)

    # Then we use fit_transform() to fit the model / learn the vocabulary,
    # then transform the data into feature vectors.
    # The input should be a list of strings. .toarraty() converts to a numpy array
    
    train_bag = vectorizer.fit_transform(X_train).toarray()
    test_bag = vectorizer.transform(X_test).toarray()

    # You can extract the vocabulary created by CountVectorizer
    # by running print(vectorizer.get_feature_names())


    print("Training the random forest classifier!\n")
    # Initialize a Random Forest classifier with 75 trees
    forest = RandomForestClassifier(n_estimators = 50) 

    # Fit the forest to the training set, using the bag of words as 
    # features and the sentiment labels as the target variable
    forest = forest.fit(train_bag, y_train)


    train_predictions = forest.predict(train_bag)
    test_predictions = forest.predict(test_bag)
    
    train_acc = metrics.accuracy_score(y_train, train_predictions)
    valid_acc = metrics.accuracy_score(y_test, test_predictions)
    print("The training accuracy is: ", train_acc, "\n", "The validation accuracy is: ", valid_acc)
    print()
    print()
    #Extract feature importnace
    print('TOP TEN IMPORTANT FEATURES:')
    importances = forest.feature_importances_
    indices = np.argsort(importances)[::-1]
    top_10 = indices[:10]
    print([vectorizer.get_feature_names()[ind] for ind in top_10])

In [ ]:
predict_sentiment(X,Y)

In [ ]:
#Sample incoming email extracted content
#In an actual deployed program, the email library and .get_payload() function can be used to extract
#content from the incoming email.
ems = "Hi Maverick! We are interested in buying your product. Kindly send us the quotes. Thank you"

In [ ]:
em = pd.DataFrame(index = range(0,1), columns=['email'])
em['email'] = ems
for i in range(0,1):
    em.iloc[i,-1] = email_cleaner(em.iloc[i,-1])

In [ ]:
forest = RandomForestClassifier(n_estimators = 50)
vectorizer = CountVectorizer(analyzer = "word",   \
                                 tokenizer = None,    \
                                 preprocessor = None, \
                                 stop_words = None,   \
                                 max_features = 2000) 

train_bag = vectorizer.fit_transform(X).toarray()
forest = forest.fit(train_bag, Y)
#ems = "Hi John! We are interested in buying your product. Kindly send us the quotes."
em_bag = vectorizer.transform(em).toarray()
sentiment = forest.predict(em_bag)

In [ ]:
sentiment[0]

In [ ]:
if (sentiment[0] == 1.0):
    print("Hi Maria,\nIt’s great to hear about your interest. Please find the quotes attached with the email. \
\nLooking forward to hearing back from you. \nThank You")
elif (sentiment[0] == 2.0):
    print('Hi Maria,\nThank you for the information. I will add that on our records. \nRegards')
elif (sentiment[0] == 3.0):
    print('Hi Maria,\nWe appologise for the misunderstanding caused. You are a valuable customer for us and\
    we would certainly like to meet and discuss the matter with you further. \nThank you')
elif (sentiment[0] == 4.0):
    print('Hi Maria,\nPlease find the attached files for the information you requested.\nThank you')